In [29]:
results_dir = 'results_tmp'
import os
os.makedirs(results_dir, exist_ok=True)

In [30]:
from experiment_info import samples, data_dir, puffs, params

In [31]:
# import custom functions
import functions as fn

print(f'data directory: {data_dir}')
print(f'Number of samples: {len(samples)}')
num_odors = len(puffs)

print(f'Number of odors: {num_odors}')
print(f'x,y,z dimensions:', params['x_dim'], params['y_dim'], params['z_dim'])
print(f'Number of frames to analyze:', params['n_frames_to_analyze'])
print(f'Number of initial frames for df/f normalization:', params['background_frames'])

data directory: /mnt/cup/labs/mcbride/bjarnold/new_analysis/data/Mar_22_2024/1_RegisteredBrains
Number of samples: 15
Number of odors: 72
x,y,z dimensions: 128 128 24
Number of frames to analyze: 112
Number of initial frames for df/f normalization: 20


In [32]:

import glob
import pandas as pd
from collections import defaultdict
import pickle

In [33]:
######
# These functions take in mean activity traces and output summary statistics or modified traces
######

def max_activity_per_odor(mean_activity, puffs, n_frames_to_analyze):

    maxs_per_odor = defaultdict(list)
    argmaxs_per_odor = defaultdict(list) # this is to find the frame with maximum activity
    
    for i,puff in enumerate(puffs):
        odor_name = puff.odor_name
        # since a subset of videos were concatenates, use index i of odor_of_interest_indices to get the corresponding interval
        interval = mean_activity[i*n_frames_to_analyze : (i+1)*n_frames_to_analyze]

        maxs_per_odor[odor_name].append(np.max(interval))
        argmaxs_per_odor[odor_name].append(np.argmax(interval))

    for odor in maxs_per_odor:
        assert len(maxs_per_odor[odor]) <= 2, f" for odor {odor} there were more than 2 trials. This is unexpected."
        assert len(argmaxs_per_odor[odor]) <= 2, f" for odor {odor} there were more than 2 trials. This is unexpected."

    return maxs_per_odor, argmaxs_per_odor

def activity_auc_per_odor(mean_activity, puffs, n_frames_to_analyze, background_frames):

    aucs_per_odor = defaultdict(list)
    # for each odor, extract max value from the corresponding interval
    for i, puff in enumerate(puffs):
        # get odor name
        odor_name = puff.odor_name

        # since a subset of videos were concatenates, use index i of odor_of_interest_indices to get the corresponding interval
        interval = mean_activity[i*n_frames_to_analyze : (i+1)*n_frames_to_analyze]

        # baseline= 1.96*np.std(interval[0:background_frames])
        baseline= np.median(interval[0:background_frames])
        peak_coord = np.argmax(interval)

        # get all indices of interval where activity is above baseline
        above_baseline = np.where(interval > baseline)[0]
        # find sets of indices in above_baseline that are consecutive, if not consecutive, then activity dropped below baseline
        not_consecutive = np.where(np.diff(above_baseline) != 1)[0]
        # an extra 1 is added to account for fact that np.diff returns array that is 1 element shorter than the input array, so add 1 to split at correct indices
        split_indices = np.split(above_baseline, not_consecutive+1)

        # find interal that contains the peak index
        for s in split_indices:
            if peak_coord in s:
                start = s[0]  # indices are for where activity is above baseline, so start 1 before
                end = s[-1] + 1 # indices are for where activity is above baseline, so end 1 after, and add extra 1 to include the last index
                peak_interval = interval[start:end]
                break

        # calculate area under the curve
        auc = np.trapz(peak_interval, dx=1)
        aucs_per_odor[odor_name].append(auc)

        # if test and i == 4 and self.name == list(activity_traces.keys())[0]:
        #     plt.plot(interval)
        #     plt.axhline(baseline, color='black', linestyle='--')
        #     plt.axvline(peak_coord, color='red')
        #     plt.axvline(start, color='green')
        #     plt.axvline(end, color='green')
        #     print(auc)

        for odor in aucs_per_odor:
            assert len(aucs_per_odor[odor]) <= 2, f" for odor {odor} there were more than 2 trials. This is unexpected."

        return aucs_per_odor
    
def subtract_paraffin_response(mean_activity, puffs, n_frames_to_analyze):

    num_odors = len(puffs)

    # find indices of the odors delivered that correspond to paraffin, should be 2 for 2 trials
    paraffin_indices = [p.number for p in puffs if p.odor_name == "paraffin"]
    assert len(paraffin_indices) == 2, f"Expected to find 2 paraffin trials, but found {len(paraffin_indices)}"
    assert paraffin_indices[0] < paraffin_indices[1], f"Expected paraffin trial 1 to come before paraffin trial 2, but found {paraffin_indices}"

    new_traces = []
    # collect paraffin traces for this sample, for both trials; in paraffin_traces dict, keys are trials
    paraffin_traces = {}
    for i, index in enumerate(paraffin_indices):
        interval = mean_activity[index*n_frames_to_analyze:(index+1)*n_frames_to_analyze]
        paraffin_traces[i] = interval
    
    # subtract paraffin traces from each odor trace
    for puff in puffs:
        i = puff.number
        interval = mean_activity[i*n_frames_to_analyze:(i+1)*n_frames_to_analyze]
        # if this odor was delivered in the first half, subtract paraffin trace from first trial
        if i <= (num_odors/2) - 1:
            interval_subtracted = interval - paraffin_traces[0]
        else:
            interval_subtracted = interval - paraffin_traces[1]
        new_traces.extend(interval_subtracted)

    mean_activity_paraffin_subtracted = np.array(new_traces)
    return mean_activity_paraffin_subtracted

In [34]:
import skimage as ski
import numpy as np
import caiman as cm
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns

class Brain:

    # This class stores data and preprocessing methods
    #
    # name: name of the sample
    # vid_fnames: the file paths of all the individual videos
    # video: the full video, a concatenation of all individual videos
    # binary_mask: 3D outline of the antennal lobe, region in video of higher pixel intensities according to otsu thresholding
    # binary_mask_frac: the fraction of pixels contained within binary_mask, compared to the entire 3d volume
    # mean_activity: the mean pixel intensity wtihin binary mask, over time; this is df_f if videos have been normalized against background activity
    # mean_activity_paraffin_subtracted: same as mean_activity except the signal of the paraffin odor has been subtracted
    # maxs_per_odor: a dictionary where keys are names of odors, values are lists of size 2, one for each trial in which the odor was administered
    # aucs_per_odor: same as maxs_per_odor, except calculating the area under the curve

    def __init__(self, name, vid_fnames):
        self.name = name
        self.vid_fnames = vid_fnames

    def __repr__(self):
        return f"Brain for {self.name} containing {len(self.vid_fnames)} videos"
    
    def reshape_vid(self, V, x_dim, y_dim, z_dim):
        # reshapes a video matrix into 4 dimensions: (n_frames, x, y, z)
        try:
            # 1st dimension of Y is time X z_stacks, so reshape according to specified z_stacks
            V_reshaped = np.reshape(V, (int(V.shape[0]/z_dim), z_dim, x_dim, y_dim))
        except ValueError:
            print(f"V.shape: {V.shape}, x_dim: {x_dim}, y_dim: {y_dim}, z_dim: {z_dim}")
            raise ValueError("The dimensions of the video are not compatible with the specified x,y,z dimensions. Please check the dimensions of the video and the x,y,z dimensions specified in experiment_info.py")
        # transpose to (n_frames, x, y, z)
        V_reshaped2 = np.transpose(V_reshaped, (0, 2, 3, 1))
        return V_reshaped2

    def background_normalize(self, V, background_frames, offset=1000):
        # normalize pixel intensity according to the mean of the first background_frames
        # the default of offset=1000 was taken from Martin's code!
        # V has shape (frames, x,y,z), first `background_frames` frames are background
        V = V + offset
        background = np.mean(V[0:background_frames,:,:,:], axis=0) # get mean of each pixel for 1st 20 frames
        V_normalized = (V - background)/(background+0.0000000000000000000001) # subtract and divide by background
        return V_normalized

    def load_videos(self, params, normalize=False):
        # takes video file names in vid_fnames and loads them into list of CaImAn movie objects
        x_dim, y_dim, z_dim = params['x_dim'], params['y_dim'], params['z_dim']
        n_frames_to_analyze = params['n_frames_to_analyze']
        background_frames = params['background_frames']
        
        vid_list = []
        for v_fname in self.vid_fnames:
            V = cm.load(v_fname)
            V = self.reshape_vid(V, x_dim, y_dim, z_dim)
            assert V.shape[0] >= n_frames_to_analyze, f"Number of frames in video is less than n_frames_to_analyze. Please specify a smaller number of frames to analyze."
            # only analyze first 'n_frames_to_analyze' frames
            V = V[:n_frames_to_analyze] 
            if normalize:
                V = self.background_normalize(V, background_frames)
            vid_list.append(V)
            
        self.video = cm.concatenate(vid_list)

    def find_binary_mask(self, params, results_dir):
        # uses scikit image to find binary mask from movie object
        x_dim, y_dim, z_dim = params['x_dim'], params['y_dim'], params['z_dim']
        # create single volume by taking the mean across frames
        V_smoothed = np.mean(self.video, axis=0)
        # spatial smoothing, median filter across pixels
        V_smoothed = ski.filters.median(V_smoothed, behavior='ndimage')
        # thresholding
        thresh = ski.filters.threshold_otsu(V_smoothed)

        self.binary_mask = V_smoothed > thresh 
        self.binary_mask_frac = np.sum(self.binary_mask)/(x_dim*y_dim*z_dim)

        self.save_and_plot_binary_mask(params, results_dir)
    
    def save_and_plot_binary_mask(self, params, results_dir):
     
        mask_dir = f'{results_dir}/binary_masks'
        plot_dir = f'{results_dir}/binary_mask_plots'

        os.makedirs(mask_dir, exist_ok=True)
        os.makedirs(plot_dir, exist_ok=True)

        # save binary mask
        with open(f'{mask_dir}/{self.name}.pkl', 'wb') as f:
            pickle.dump(self.binary_mask, f)

        # plot binary masks
        colors = [(0, 0, 0, 0), (0, 0, 0, 0.1)]  # RGBA tuples, the 1st color's alpha set to 0 to make transparent so white values are ignores, black values are partially transparent; 0's mapped to 1st color, 1's mapped to 2nd color
        cmap = ListedColormap(colors)

        fig, ax = plt.subplots(1,1, figsize=(3,3))
        for i in range(0, params['z_dim']):
            plt.imshow(self.binary_mask[:,:,i], cmap=cmap)
        plt.title(self.name)
        sns.despine()
        plt.savefig(f'{plot_dir}/{self.name}.png', dpi=300)
        plt.close()

    def mean_activity_within_binary_mask(self, params):
        # using pixels contained in binary_mask, measure the mean pixel intensity over time

        x_dim, y_dim, z_dim = params['x_dim'], params['y_dim'], params['z_dim']
        # vectorize both the binary mask and the video
        binary_mask_reshaped = np.reshape(self.binary_mask, (x_dim*y_dim*z_dim))
        video_reshaped = np.reshape(self.video, (self.video.shape[0], -1))
        video_reshaped = np.array(video_reshaped)
        print(binary_mask_reshaped.shape, video_reshaped.shape)

        self.mean_activity = np.mean(video_reshaped[:,binary_mask_reshaped], axis=1)

    def subtract_paraffin(self, puffs, params):
        
        n_frames_to_analyze = params['n_frames_to_analyze']
        self.mean_activity_paraffin_subtracted = subtract_paraffin_response(self.mean_activity, puffs, n_frames_to_analyze)

    def compute_max_responses(self, puffs, params):
        # this is a wrapper function for 'max_activity_per_odor' which can also be used on paraffin subtracted activity

        n_frames_to_analyze = params['n_frames_to_analyze']
        maxs_per_odor, argmaxs_per_odor = max_activity_per_odor(self.mean_activity, puffs, n_frames_to_analyze)

        self.maxs_per_odor = maxs_per_odor
        self.argmaxs_per_odor = argmaxs_per_odor

    def compute_AUC(self, puffs, params, test=False):
        n_frames_to_analyze = params['n_frames_to_analyze']
        background_frames = params['background_frames']

        aucs_per_odor = activity_auc_per_odor(self.mean_activity, puffs, n_frames_to_analyze, background_frames)

        self.aucs_per_odor = aucs_per_odor

In [35]:
# the brains list will store instances of the Brain class that will keep track of data and preprocessing methods
brains = []

for samp in samples:
    v = glob.glob(f"{data_dir}/{samp}/*.registered.tif")
    v = sorted(v)
    num_vids = len(v)
    assert num_vids == len(puffs), f"I found {num_vids} videos for sample {samp}, but there are {len(puffs)} odors. The number of videos and odors should be equivalent."
    
    brain = Brain(samp, v)
    brains.append(brain)


In [36]:
for i,b in enumerate(brains):
    if i != 0:
        continue
    # using filenames of videos, load them into movie objects, creating video instance attribute
    b.load_videos(params, normalize=False)
    # use otsu thresholding to find binary mask, specify a results dir to save the binary mask object and plot
    b.find_binary_mask(params, results_dir)

    # use the binary mask to compute mean activity over time
    # except here we will re-load the videos, overwriting self.video, and normalize them against the first params['n_frames_to_analyze']
    b.load_videos(params, normalize=True)
    b.mean_activity_within_binary_mask(params)
    b.subtract_paraffin(puffs, params)


    

(393216,) (8064, 393216)


In [37]:
for i,b in enumerate(brains):
    if i != 0:
        continue
    b.compute_max_responses(puffs, params)
    b.compute_AUC(puffs, params)

## save the mean trace activity as csv file

In [38]:
mean_activity = {b.name : b.mean_activity for i,b in enumerate(brains) if i == 0}
mean_activity_df = pd.DataFrame.from_dict(mean_activity)
mean_activity_df.to_csv(f'{results_dir}/mean_activity_within_mask.csv', index=False)

## save maxs per odor as dataFrame

In [39]:
def convert_to_df(brains, puffs, metric):
    df_list = []
    for i,b in enumerate(brains):
        if i != 0:
            continue
        df_tmp = pd.DataFrame.from_dict(getattr(b, metric))
        df_tmp['samp'] = b.name
        df_tmp['subpop'] = b.name.split('_')[1]
        df_tmp['trial'] = df_tmp.index+1
        df_list.append(df_tmp)
    df = pd.concat(df_list)
    df = df.reset_index(drop=True)
    df = pd.melt(df, id_vars=['samp', 'subpop', 'trial'], var_name='odor', value_name='value')

    odor_order = {}
    for puff in puffs:
        if puff.trial == 1:
            odor_order[puff.odor_name] = puff.number

    df['odor_order'] = df['odor'].map(odor_order)
    return df

peak_max_df = convert_to_df(brains, puffs, 'maxs_per_odor')
peak_auc_df = convert_to_df(brains, puffs, 'aucs_per_odor')
peak_max_df


,samp,subpop,trial,odor,value,odor_order
0,230913_ORL_GCaMP6f_F1,ORL,1,hexanal4.375,0.000476,0
1,230913_ORL_GCaMP6f_F1,ORL,2,hexanal4.375,0.000475,0
2,230913_ORL_GCaMP6f_F1,ORL,1,hexanal4.03,0.001123,1
3,230913_ORL_GCaMP6f_F1,ORL,2,hexanal4.03,0.000477,1
4,230913_ORL_GCaMP6f_F1,ORL,1,hexanal2.825,0.001073,2
...,...,...,...,...,...,...
67,230913_ORL_GCaMP6f_F1,ORL,2,hexanoic acid,0.001025,33
68,230913_ORL_GCaMP6f_F1,ORL,1,camphor,0.009764,34
69,230913_ORL_GCaMP6f_F1,ORL,2,camphor,0.006896,34
70,230913_ORL_GCaMP6f_F1,ORL,1,1-octen-3-ol,0.000644,35


In [40]:
def make_mean_activity_plot(brains, params, metric):

    fig, axs = plt.subplots(1, 1, figsize=(16, 4))

    for i,b in enumerate(brains):
        if i != 0:
            continue

        activity = getattr(b, metric)
        plt.plot(activity + i*0.02, c='black')  # Offset each trace by i*3
        # print sample name on the right
        plt.text(len(activity)*1.02, i*0.02, b.name, color='black')

    # print the names of the odors on the x-axis
    odor_names = []
    positions = []
    for i,puff in enumerate(puffs):
        odor_names.append(puff.odor_name)
        positions.append(i*params['n_frames_to_analyze'] + params['n_frames_to_analyze']/2)
    plt.xticks(positions, odor_names, rotation=90)

    # draw vertical lines to separate odors
    for i in range(len(puffs)):
        plt.axvline((i+1)*params['n_frames_to_analyze'], color="black", linestyle="--", alpha=0.1)

    plt.yticks([])
    # supress grid lines
    plt.grid(False)
    sns.despine()

    plt.savefig(f'{results_dir}/{metric}.png', dpi=300)
    plt.close()

make_mean_activity_plot(brains, params, metric='mean_activity')
make_mean_activity_plot(brains, params, metric='mean_activity_paraffin_subtracted')